In [49]:
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

In [23]:
rf_info = pd.read_csv("Risk_free_rate.csv", parse_dates = ["Date"], index_col = "Date")
# calculate the product once and pass it as an argument to the function
rf_cumprod = (1+rf_info/100).cumprod() 
rf_date, cumprod_rf  =  rf_info.index, rf_cumprod.values.tolist()

In [21]:
def jensen_alpha(start_date,rf_date,cumprod_rf,rp, rm , B):
    # Find date in rf that is greater than or equal to trading date
    idx = rf_date.searchsorted(start_date)
    end_idx = rf_date.searchsorted(start_date+ timedelta(days= 365 ) )
    try:
        if idx < end_idx:
            rf = cumprod_rf[end_idx -1][1] / cumprod_rf[idx-1][1] -1
            rm_rf = rm - rf
           
        else:
            rf = cumprod_rf[idx][1] / cumprod_rf[idx - 1][1] -1
            rm_rf =  rm - rf
    except IndexError:
        return None
    
    # rp is percent return
    # b is beta
    if(rp == -999 or B == -999):
        return None
    #print(rp,rf,B, rm_rf)
    return rp - rf - B * rm_rf

In [22]:
def simple_alpha(rp,rm):
    return rp - rm

In [68]:
OUTPUTFILE = 'outputs_19_with_alpha7_3.csv'

In [75]:
alpha_df

,Unnamed: 0,tikr,Date,label,pred,score,year,k,Release Date,Outlook,Percent Return,Annual Return,beta,sp Annual,sp Percent,jensen alpha (7),simple alpha (7)
0,16544068,CPRI,2018-12-31,0,1,0.718268,2018,1,2018-12-31,0,0.062236,22.344344,0.825263,5.272835,0.035818,0.032606,0.041081
1,16544079,RHP,2018-12-31,1,1,0.735506,2018,1,2018-12-31,1,0.127005,511.183578,0.794646,5.272835,0.035818,0.098459,0.105850
2,16544091,ARCH,2018-12-31,1,1,0.710367,2018,1,2018-12-31,1,0.036993,5.655043,0.770988,5.272835,0.035818,0.009284,0.015837
3,16544093,NFLX,2018-12-31,1,1,0.846352,2018,1,2018-12-31,1,0.212957,23708.832092,1.850919,5.272835,0.035818,0.147010,0.191801
4,16544105,LNC,2018-12-31,1,1,0.702053,2018,1,2018-12-31,1,0.078589,50.804560,1.210388,5.272835,0.035818,0.035321,0.057433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97183,17517072,SMPL,2022-01-05,1,1,0.762264,2022,5,2022-01-05,1,0.006686,0.415783,1.160258,-0.343288,-0.008027,0.015999,0.035635
97184,17517074,FNF,2022-01-05,0,1,0.750397,2022,5,2022-01-05,0,0.069897,32.963511,1.005114,-0.343288,-0.008027,0.077964,0.098846
97185,17517177,ALNY,2022-01-04,0,1,0.749944,2022,5,2022-01-04,0,-0.104279,-0.996805,0.703093,-0.773521,-0.028061,-0.084550,-0.073968
97186,17517210,BX,2022-01-03,0,1,0.709770,2022,5,2022-01-03,0,-0.033878,-0.834428,1.118325,-0.536715,-0.014638,-0.017509,-0.010646


In [69]:
alpha_df = pd.read_csv(OUTPUTFILE)
alpha_df = alpha_df[alpha_df['sp Annual'] != -999]
alpha_df = alpha_df[alpha_df['beta'] >= 0]
alpha_df = alpha_df[alpha_df['beta'] <= 3.0]

In [70]:
grouped_df = alpha_df.groupby(['year', 'k']).mean()
year_averages = grouped_df.groupby(['year']).mean()
year_averages

/var/folders/nj/5crsgjwx3sxd51h31pmwlxj80000gn/T/ipykernel_72178/1188986458.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  grouped_df = alpha_df.groupby(['year', 'k']).mean()


,Unnamed: 0,label,pred,score,Outlook,Percent Return,Annual Return,beta,sp Annual,sp Percent,jensen alpha (7),simple alpha (7)
year,,,,,,,,,,,,
2018,1.661184e+07,0.547924,1.0,0.767178,0.547924,0.003245,1.189221e+07,0.990028,1.094248,-0.001251,0.004194,0.004647
2019,1.680951e+07,0.386831,1.0,0.730490,0.386831,0.016855,1.350132e+15,0.923617,0.740559,0.005107,0.012112,0.013392
2020,1.703523e+07,0.532068,1.0,0.704909,0.532068,0.010460,3.211224e+15,1.055762,10.185770,0.006825,0.002637,0.004494
2021,1.720306e+07,0.427361,1.0,0.827348,0.427361,0.005181,9.380621e+09,1.100008,0.763796,0.004809,-0.000086,0.002520
2022,1.739736e+07,0.314158,1.0,0.789861,0.382560,0.005605,3.116008e+05,1.071786,5.058740,0.020271,-0.016402,-0.028577


In [71]:
import pandas as pd
from scipy.stats import gmean

grouped_df = alpha_df.groupby(['year', 'k'])

def calculate_gmean(group):
    return gmean(group + 1)

k_returns = grouped_df['Percent Return'].apply(calculate_gmean)
yearly_returns = k_returns.groupby('year').apply(gmean)
yearly_returns = yearly_returns - 1


k_sp_returns = grouped_df['sp Percent'].apply(calculate_gmean)
yearly_sp_returns = k_sp_returns.groupby('year').apply(gmean)
yearly_sp_returns = yearly_sp_returns - 1

# yearly_sp_returns

In [72]:
dates = ['2018-01-01', '2019-01-01', '2020-01-01', '2021-01-01', '2022-01-01']

In [73]:
def calculate_alphas(dates, rf_date, cumprod_rf, yearly_returns, yearly_sp_returns, betas):
    alphas = []
    jensen_alphas = []
    
    for i in range(len(dates)):
        date = datetime.strptime(dates[i], '%Y-%m-%d')
        jen = jensen_alpha(date, rf_date, cumprod_rf, yearly_returns[i], yearly_sp_returns[i], betas[i])
        sim = simple_alpha(yearly_returns[i], yearly_sp_returns[i])
        
        jensen_alphas.append(jen)
        alphas.append(sim)
    
    data_dict = {"Year": dates, "Percent Returns": yearly_returns,
                 "sp Percent": yearly_sp_returns,
                 "beta": betas,
                 "Simple Alpha": alphas,
                 "Jensen Alpha": jensen_alphas}
    return pd.DataFrame(data_dict)

In [74]:
calculate_alphas(dates, rf_date, cumprod_rf, list(yearly_returns), 
                 list(yearly_sp_returns), list(year_averages['beta']))
                 

,Year,Percent Returns,sp Percent,beta,Simple Alpha,Jensen Alpha
0,2018-01-01,0.001133,-0.001663,0.990028,0.002796,0.002599
1,2019-01-01,0.015065,0.004981,0.923617,0.010084,0.008815
2,2020-01-01,0.005339,0.005787,1.055762,-0.000448,-0.000550
3,2021-01-01,0.002662,0.004689,1.100008,-0.002027,-0.002496
4,2022-01-01,0.003250,0.019676,1.071786,-0.016425,-0.016813
